In [3]:
import torch
from torch import nn
import numpy as np
import torchvision
import torchvision.transforms as transforms
import matplotlib.pyplot as plt
from PIL import Image

In [35]:
path_youwu = "F:\OneDrive\AI资料\机器学习\浩杰给的课件\pytorch\实验数据\去雾数据集\有雾图片"
path_quwu = "F:\OneDrive\AI资料\机器学习\浩杰给的课件\pytorch\实验数据\去雾数据集\去雾图片"

def readImagesToNumpy(path):
    x = []
    for filename in os.listdir(path, ):
        #print(filename)
        file = path + '\\' + filename
        img = Image.open(file)
        img = img.resize((200, 100), Image.ANTIALIAS) #read the image
        img = np.array(img) # transform to numpy
        img = img / 255 # normalize
        #plt.imshow(img)
        #print(img.shape)
        x.append(img)
    return x

X = readImagesToNumpy(path_youwu)
y = readImagesToNumpy(path_quwu)
X = torch.tensor(X)
y = torch.tensor(y)
print(X.shape, y.shape)

torch.Size([520, 100, 200, 3]) torch.Size([520, 100, 200, 3])
